In [1]:
from functools import wraps

def typed(*type_args, **type_kwargs):
    def dec(fn):
        @wraps(fn)
        def wrap(*args, **kwargs):
            for i, t in enumerate(type_args):
                if not isinstance(args[i], t):
                    print('pos {} argument {} type error'.format(i, args[i]))
                    return None
            for k, t in type_kwargs.items():
                if not isinstance(kwargs[k], t):
                    print('keyword argument {} => {} type error'.format(k, kwargs[k]))
                    return None
            return fn(*args, **kwargs)
        return wrap
    return dec

In [2]:
@typed(x=int, y=int)
def add(x, y):
    return x + y

In [3]:
add(1, 2)

KeyError: 'x'

In [4]:
add(x=1, y=2)

3

In [5]:
add(x=1, y='2')

keyword argument y => 2 type error


In [6]:
import inspect

In [7]:
def add(x: int, y: int) -> int:
    return x + y

In [8]:
help(inspect.signature)

Help on function signature in module inspect:

signature(obj, *, follow_wrapped=True)
    Get a signature object for the passed callable.



In [10]:
sig = inspect.signature(add)

In [19]:
for v in sig.parameters.values():
    print(v.annotation)

POSITIONAL_OR_KEYWORD
POSITIONAL_OR_KEYWORD


In [23]:
def typed(fn):
    @wraps(fn)
    def wrap(*args, **kwargs):
        for i, param in enumerate(inspect.signature(fn).parameters.values()):
            if param.annotation:
                if param.name in kwargs.keys():
                    if not isinstance(kwargs[param.name], param.annotation):
                        raise TypeError(param.name)
                else:
                    if not isinstance(args[i], param.annotation):
                        raise TypeError(param.name)
        return fn(*args, **kwargs)
    return wrap

In [24]:
@typed
def add(x: int, y: int):
    return x + y

In [25]:
add(1, 2)

3

In [26]:
add('1', 2)

TypeError: x

In [27]:
add(x=1, y=2)

3

In [28]:
add(x=1, y='2')

TypeError: y

In [36]:
def partial(fn, *p_args, **p_kwargs):
    @wraps(fn)
    def wrap(*args, **kwargs):
        return fn(*p_args, *args, **p_kwargs, **kwargs)
    return wrap

In [33]:
def add(x, y):
    return x + y

In [37]:
inc = partial(add, y=1)

In [38]:
inc(3)

4

In [39]:
def dict_eq(a, b):
    for k, v in a.items():
        if k not in b.keys():
            return False
        else:
            if not isinstance(v, (dict, list)):
                if v != b[k]:
                    return False
            if isinstance(v, list):
                for i, item in enumerate(v):
                    if len(b[k]) <= i:
                        return False
                    if b[k][i] != item:
                        return False
            if isinstance(v, dict):
                ret = dict_eq(v, b[k])
                if ret is False:

                    return ret

In [40]:
import random
import time

def data_source():
    while True:
        yield random.randint(0, 100)
        time.sleep(0.1)

In [41]:
ds = data_source()

In [42]:
for _ in range(10):
    print(next(ds))

53
23
80
79
98
73
23
93
61
79


In [43]:
import datetime

In [44]:
t1 = datetime.datetime.now()

In [45]:
t1

datetime.datetime(2016, 8, 6, 9, 23, 30, 910460)

In [46]:
t2 = datetime.datetime.now()

In [48]:
diff = (t2 - t1)

In [50]:
diff.total_seconds()

25.857265

In [56]:
import datetime

def top_k1(k, time=3):
    start = datetime.datetime.now()
    lst = []
    while True:
        lst.append(next(ds))
        current = datetime.datetime.now()
        if (current - start).total_seconds() >= time:
            start = current
            lst.sort()
            ret = []
            for _ in range(k):
                ret.append(lst.pop())
            yield ret
                

In [52]:
g = top_k1(10)

In [54]:
for _ in range(3):
    print(next(g))

[78, 77, 74, 73, 72, 70, 69, 67, 67, 66]
[99, 95, 91, 90, 88, 84, 78, 74, 73, 69]
[97, 92, 89, 88, 85, 83, 66, 65, 63, 63]


In [60]:
import datetime

def top_k2(k, time=3):
    start = datetime.datetime.now()
    lst = []
    while True:
        #lst.append(next(ds))
        e = next(ds)
        for i, v in enumerate(lst):
            if e < v:
                lst.insert(i, e)
                break
        else:
            lst.append(e)
        current = datetime.datetime.now()
        if (current - start).total_seconds() >= time:
            start = current
            #lst.sort()
            ret = []
            for _ in range(k):
                ret.append(lst.pop())
            yield ret

In [61]:
g2 = top_k2(10)

In [62]:
for _ in range(3):
    print(next(g2))

[92, 90, 90, 88, 86, 84, 77, 72, 67, 59]
[98, 92, 91, 91, 89, 89, 80, 79, 78, 75]
[99, 95, 88, 88, 84, 81, 80, 77, 77, 72]


In [63]:
for x in range(10):
    print(random.randint(0, 100))

48
98
47
97
31
41
64
65
59
68


In [75]:
def heap():
    data = []
    def add(e):
        idx = len(data)
        data.append(e)
        parent_idx = (idx - 1) // 2
        while parent_idx >= 0:
            if data[idx] > data[parent_idx]:
                data[parent_idx], data[idx] = data[idx], data[parent_idx]
                idx = parent_idx
                parent_idx = (idx - 1) // 2
            else:
                break
    
    def pop():
        if not data:
            return None
        if len(data) == 1:
            return data.pop()
        idx = 0
        ret = data[idx]
        data[idx] = data.pop()
        left_idx = 2 * idx + 1
        rigth_idx = left_idx + 1
        while left_idx < len(data):
            child_idx = left_idx
            if rigth_idx < len(data) and data[rigth_idx] > data[left_idx]: # 存在右子节点 并且 右子节点大于左子节点
                child_idx = rigth_idx
            if data[idx] < data[child_idx]:
                data[idx], data[child_idx] = data[child_idx], data[idx]
                idx = child_idx
                left_idx = 2 * idx + 1
                rigth_idx = left_idx + 1
            else:
                break
        return ret
    
    return add, pop
            
        

In [76]:
add, pop = heap()

In [77]:
for _ in range(10):
    add(random.randint(0, 100))

In [78]:
for _ in range(10):
    print(pop())

97
93
68
45
39
33
32
25
19
0


In [80]:
import datetime

def top_k3(k, time=3):
    start = datetime.datetime.now()
    add, pop = heap()
    while True:
        add(next(ds))
        current = datetime.datetime.now()
        if (current - start).total_seconds() >= time:
            start = current
            ret = []
            for _ in range(k):
                ret.append(pop())
            yield ret

In [84]:
g3 = top_k3(10)
for _ in range(3):
    print(next(g3))

[100, 99, 97, 97, 85, 81, 80, 75, 68, 66]
[97, 97, 96, 88, 88, 88, 87, 86, 85, 83]
[93, 92, 92, 91, 83, 81, 81, 79, 79, 75]
